<a href="https://colab.research.google.com/github/obystephen/Phase-2-Project-/blob/main/eCommerce_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing Libraries

**Text Preprocessing.**

In [134]:
# import re
# from sklearn.preprocessing import StandardScaler, LabelEncoder

# # Text Preprocessing
# # Convert text to lowercase
# selected_df_copy['Text'] = selected_df_copy['Text'].str.lower()

# # Remove punctuation
# selected_df_copy['Text'] = selected_df_copy['Text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# # Tokenization (if needed, Rasa NLU can handle this)
# # No explicit tokenization is needed as Rasa NLU handles tokenization internally

# # Feature scaling (if needed)
# # Standardize numerical columns using StandardScaler
# scaler = StandardScaler()
# numerical_columns = ['Alignment Score', 'Agreement', 'Length', 'Error']
# selected_df_copy[numerical_columns] = scaler.fit_transform(selected_df_copy[numerical_columns])

# # Now, your dataset is preprocessed and feature-engineered, ready for training Rasa models.
# # Ensure that you split your dataset into training and testing sets before training your models.


In [135]:
# # Display the first few rows of the DataFrame
# print(selected_df_copy.head())

# # Check data types of columns
# print(selected_df_copy.dtypes)

# # Check unique values and counts for encoded categorical variables
# for col in categorical_columns:
#     print(f"Unique values for {col}:")
#     print(selected_df_copy[col].value_counts())

# # Check mean and standard deviation of standardized numerical columns
# print(selected_df_copy[numerical_columns].describe())

# # Plot histogram of 'Alignment Score' after standardization
# import seaborn as sns
# import matplotlib.pyplot as plt

# plt.figure(figsize=(8, 6))
# sns.histplot(selected_df_copy['Alignment Score'], bins=20, kde=True)
# plt.xlabel('Alignment Score (Standardized)')
# plt.ylabel('Frequency')
# plt.title('Distribution of Alignment Score')
# plt.show()


# **MODELLING.**

**Setting up RASA Environment.**

In [136]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

In [137]:
# from google.colab import drive
# drive.mount('/gdrive')

In [138]:
#checking python version.
!python3 --version
!pip3 --version

Python 3.10.12
pip 24.0 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [139]:
# Install python3-venv package
!apt install python3.10-venv




Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10-venv is already the newest version (3.10.12-1~22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.


In [140]:
# Create a virtual environment
!python3 -m venv ./venv

# Activate the virtual environment
!source ./venv/bin/activate


Installing RASA Open source.

In [141]:
#Ensuring pip version is up to date.
!pip3 install -U pip

In [142]:
#installing RASA open source.
!pip3 install rasa

Spacy Language Model.

In [143]:
!pip3 install 'rasa[spacy]'
!python3 -m spacy download en_core_web_md
!pip install --upgrade spacy
!python -m spacy download en_core_web_sm


  Using cached spacy-3.4.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.4.4 which is incompatible.
2024-02-12 21:43:34.653670: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 21:43:34.703984: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-12 21:43:34.704754: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following 

In [144]:
#print(selected_df_copy.columns)

Splitting the data for modelling

In [145]:
# from sklearn.model_selection import train_test_split

# # Define features (X) and target (y)
# X = selected_df_copy[['Text', 'MergedSelections', 'Alignment Score', 'Agreement', 'Other', 'Express Emotion', 'Length', 'Error']]
# y = selected_df_copy[['Greeting', 'Backstory', 'Rant', 'Gratitude', 'Justification']]  # Include all sentiment categories

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Check the shape of the training and testing sets
# print("Training set shape:", X_train.shape, y_train.shape)
# print("Testing set shape:", X_test.shape, y_test.shape)



In [146]:
# # Iterate through the 'Text' column and print some information
# for text in selected_df_copy['Text'].sample(100):  # Sample 5 random texts
#     print("Text:", text)
#     print("-" * 50)


**RASA NLU Training Data.**

**nlu.yml.**

In [147]:
version="3.0"

nlu_md = """
## intent:greet
- hey
- hello there
- hi
- good morning
- good evening
- morning
- hey there
- let's go
- hey buddy
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- see you
- good by
- see you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later
-see yaa

## intent:positive
- yes
- indeed
- of course
- that sounds good
- correct
-exactly

## intent:negative
- no
- never
- I don't think so
- don't like that
- no way
- not really
-not at all

## intent:approving
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect
-looks good to me
-sounds good to me

## intent:disapproving
- my day was horrible
- I don't feel very well
- I am disappointed
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good



## intent: inform
- A [hair care product](service:beauty)
- [hair care product](service:beauty)
- [makeup brush](service:beauty)
- a [skincare routine](service:beauty)
- [booking](service:spa)
- a [booking](service:spa)
- a [service](service:spa)
- of a [makeup session](service:beauty)
- of a [service](service:spa)
- a [booking](service:spa), please
- a [hair styling appointment](service:beauty), please

## intent: query
- what assistance can you provide
- how may you be of help
- what services do you offer
- what support can I expect from you
- what help is available through your platform
"""

with open("nlu.md", "w") as file:
    file.write(nlu_md)
print("Output has been written to nlu.md file.")



Output has been written to nlu.md file.


**config.yml.**

In [148]:
version="3.0"
config_yml = """
language: "en"
pipeline:
  - name: "WhitespaceTokenizer"
  - name: "RegexFeaturizer"
  - name: "LexicalSyntacticFeaturizer"
  - name: "CountVectorsFeaturizer"
  - name: "CountVectorsFeaturizer"
    analyzer: "char_wb"
    min_ngram: 1
    max_ngram: 4
  - name: "DIETClassifier"
    epochs: 100
  - name: "EntitySynonymMapper"
  - name: "ResponseSelector"
    epochs: 100
#  - name: "FallbackClassifier"
#    threshold: 0.3
#    ambiguity_threshold: 0.1

policies:
  - name: "MemoizationPolicy"
  - name: "TEDPolicy"
    max_history: 5
    epochs: 100
  - name: "RulePolicy"
"""

with open("config.yml", "w") as f:
    f.write(config_yml)
print("Output has been written to config.yml file.")


Output has been written to config.yml file.


**Domain.yml.**

In [150]:
version: "3.0"
domain_yml="""
intents:
  - greet
  - goodbye
  - positive
  - negative
  - approving
  - disapproving
  - inform
  - query

responses:
  utter_greet:
  - text: "Hey! How can I assist you today?"

  utter_goodbye:
  - text: "Goodbye! Have a great day ahead."

  utter_positive:
  - text: "Glad to hear that!"

  utter_negative:
  - text: "I see. Is there anything I can do to help?"

  utter_approving:
  - text: "Awesome! Let me know if there's anything specific you need."

  utter_disapproving:
  - text: "I'm sorry to hear that. How can I make things better for you?"

  utter_inform:
  - text: "Sure, I'll assist you with that."

  utter_query:
  - text: "Feel free to ask me anything."

session_config:
  session_expiration_time: 60
  carry_over_slots_to_new_session: true
  """
with open("domain.yml", "w") as f:
     f.write(domain_yml)

print("Output has been written to domain.yml file.")

Output has been written to domain.yml file.


**rules.yml.**

In [152]:
version: "3.0"
rules_yml="""
rules:
  - rule: Handle greeting
    steps:
      - intent: greet
      - action: utter_greet

  - rule: Handle goodbye
    steps:
      - intent: goodbye
      - action: utter_goodbye

  - rule: Affirmation
    steps:
      - intent: positive
      - action: utter_positive

  - rule: Deny
    steps:
      - intent: negative
      - action: utter_negative

  - rule: Handle great mood
    steps:
      - intent: approving
      - action: utter_approving

  - rule: Handle unhappy mood
    steps:
      - intent: disapproving
      - action: utter_disapproving

  - rule: Provide information
    steps:
      - intent: inform
      - action: utter_inform

  - rule: Handle query
    steps:
      - intent: query
      - action: utter_query
"""
with open("stories.yml", "w") as f:
     f.write(config_yml)

print("Output has been written to rules.yml file.")

Output has been written to rules.yml file.


**stories.yml.**

In [154]:
version: "3.0"
stories_yml="""
stories:
  - story: Greet and goodbye
    steps:
      - intent: greet
      - action: utter_greet
      - intent: goodbye
      - action: utter_goodbye

  - story: User is happy
    steps:
      - intent: positive
      - action: utter_positive

  - story: User is unhappy
    steps:
      - intent: negative
      - action: utter_negative

  - story: User affirms
    steps:
      - intent: approving
      - action: utter_approving

  - story: User denies
    steps:
      - intent: disapproving
      - action: utter_disapproving

  - story: User asks for information
    steps:
      - intent: inform
      - action: utter_inform

  - story: User queries
    steps:
      - intent: query
      - action: utter_query
"""
with open("stories.yml", "w") as f:
     f.write(stories_yml)
print("Output has been written to stories.yml file.")

Output has been written to stories.yml file.


**Training the RASA model.**

In [156]:
!rasa train --data nlu.md stories.yml

/usr/local/lib/python3.10/dist-packages/rasa/core/tracker_store.py:1044: MovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base: DeclarativeMeta = declarative_base()
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:121: DeprecationWarning: pkg_resources is deprecated as an API
  warnings.warn("pkg_resources is deprecated as an API", DeprecationWarning)
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:2870: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP